In [5]:
import pandas as pd
import numpy as np
import metrics_eval
anime_path = 'Dataset/anime.csv'
train_path = 'Dataset/train_rating.csv'
test_path = 'Dataset/test_rating.csv'

anime = pd.read_csv(anime_path)
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

Tiền xử lý dữ liệu

In [6]:
anime_processed = anime.copy()

# 1. Xử lý Missing Values: Thay thế NaN bằng "unknown"
anime_processed.fillna({'genre': 'unknown', 'type': 'unknown'}, inplace=True)

print(f"\n--- Sau khi xử lý Missing Values ---")
print(f"Số NaN còn lại trong cột 'genre': {anime_processed['genre'].isna().sum()}")
print(f"Số NaN còn lại trong cột 'type': {anime_processed['type'].isna().sum()}")

# 2. Chuẩn hóa văn bản 'genre'
# Tách các thể loại ra thành một danh sách (list)
anime_processed['genre_list'] = anime_processed['genre'].apply(lambda x: x.lower().split(', '))

# Biến các cụm từ thành từ đơn
def join_genres(genre_list):
    processed_list = []
    for genre in genre_list:
        processed_genre = genre.replace(' ', '_').replace('-', '_')
        processed_list.append(processed_genre)
    return ' '.join(processed_list)

anime_processed['genre_processed'] = anime_processed['genre_list'].apply(join_genres)

# 3. Chuẩn hóa văn bản 'type'
anime_processed['type'] = anime_processed['type'].str.lower()

# 4. Tạo cột "tags"
anime_processed['tags'] = anime_processed['genre'] + ' ' + anime_processed['type']

print("\n--- Kết quả sau khi tiền xử lý ---")
print(anime_processed[['name', 'genre', 'type', 'tags']].head())


--- Sau khi xử lý Missing Values ---
Số NaN còn lại trong cột 'genre': 0
Số NaN còn lại trong cột 'type': 0

--- Kết quả sau khi tiền xử lý ---
                               name  \
0                    Kimi no Na wa.   
1  Fullmetal Alchemist: Brotherhood   
2                          Gintama°   
3                       Steins;Gate   
4                     Gintama&#039;   

                                               genre   type  \
0               Drama, Romance, School, Supernatural  movie   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     tv   
2  Action, Comedy, Historical, Parody, Samurai, S...     tv   
3                                   Sci-Fi, Thriller     tv   
4  Action, Comedy, Historical, Parody, Samurai, S...     tv   

                                                tags  
0         Drama, Romance, School, Supernatural movie  
1  Action, Adventure, Drama, Fantasy, Magic, Mili...  
2  Action, Comedy, Historical, Parody, Samurai, S...  
3                    

Xây dựng ma trận TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Khởi tạo TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 1),
    stop_words=['unknown']
)

# 2. Huấn luyện (Fit & Transform)
tfidf_matrix = tfidf_vectorizer.fit_transform(anime_processed['tags'])


# 3. Tạo một ánh xạ: từ anime_id -> index của hàng trong tfidf_matrix
anime_id_to_index = pd.Series(
    anime_processed.index,
    index=anime_processed['anime_id']
).to_dict()

# 4. Kết quả (Item Profile Matrix)
print(f"\nĐã tạo Ma trận Hồ sơ Anime (TF-IDF Matrix) thành công!")
print(f"Loại ma trận: {type(tfidf_matrix)}")
print(f"Kích thước của ma trận (Shape): {tfidf_matrix.shape}")
print(f"  - Số hàng (Anime): {tfidf_matrix.shape[0]} (khớp với số lượng anime)")
print(f"  - Số cột (Features/Từ vựng độc nhất): {tfidf_matrix.shape[1]}")

# Hiển thị một vài features (từ vựng) đã học được để kiểm tra
try:
    feature_names = tfidf_vectorizer.get_feature_names_out()
    print(f"\nTổng số features (từ vựng) đã học: {len(feature_names)}")
    print(f"Tất cả features (từ vựng) đã học: {feature_names[:48]}")
except Exception as e:
    print(f"Không thể lấy feature names: {e}")

# Hiển thị ma trận
print("\nMa trận TF-IDF (dạng thưa) - 5 hàng, 50 cột đầu tiên:")
print(tfidf_matrix[:5, :50].toarray())


Đã tạo Ma trận Hồ sơ Anime (TF-IDF Matrix) thành công!
Loại ma trận: <class 'scipy.sparse._csr.csr_matrix'>
Kích thước của ma trận (Shape): (12294, 52)
  - Số hàng (Anime): 12294 (khớp với số lượng anime)
  - Số cột (Features/Từ vựng độc nhất): 52

Tổng số features (từ vựng) đã học: 52
Tất cả features (từ vựng) đã học: ['action' 'adventure' 'ai' 'arts' 'cars' 'comedy' 'dementia' 'demons'
 'drama' 'ecchi' 'fantasy' 'fi' 'game' 'harem' 'hentai' 'historical'
 'horror' 'josei' 'kids' 'life' 'magic' 'martial' 'mecha' 'military'
 'movie' 'music' 'mystery' 'of' 'ona' 'ova' 'parody' 'police' 'power'
 'psychological' 'romance' 'samurai' 'school' 'sci' 'seinen' 'shoujo'
 'shounen' 'slice' 'space' 'special' 'sports' 'super' 'supernatural'
 'thriller']

Ma trận TF-IDF (dạng thưa) - 5 hàng, 50 cột đầu tiên:
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.4064272  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         

Xây dựng "Hồ sơ Người dùng"
- rating 1, 2, 3, 4 (lọc bỏ)
- rating == -1: weight = 1.0 (trọng số cơ bản thể hiện có quan tâm)
- rating 5, 6: weight = 1.5
- rating 7, 8: weight = 2.0
- rating 9, 10: weight = 3.0

In [8]:
from scipy.sparse import csr_matrix, vstack

# 1. Ánh xạ Trọng số
train_filtered = train.copy()

def map_weight(rating):
    """Hàm ánh xạ rating sang trọng số (weight)."""
    if rating == -1:
        return 1.0
    if 5 <= rating <= 6:
        return 2.0
    if 7 <= rating <= 8:
        return 4.0
    if 9 <= rating <= 10:
        return 8.0
    else:
        return 0.0  # Ratings 1-4 sẽ có trọng số 0

train_filtered['weight'] = train_filtered['rating'].apply(map_weight)

# 2. Lọc dữ liệu: Chỉ giữ lại các hàng có trọng số > 0
original_rows = len(train_filtered)
train_filtered = train_filtered[train_filtered['weight'] > 0]
filtered_rows = len(train_filtered)
print(f"Đã lọc train_rating: từ {original_rows} hàng xuống còn {filtered_rows} hàng (weight > 0).")

# 3. Tính toán User Profile (Weighted Average)
user_profiles_list = []
user_ids_list = []

# Group theo user_id để xử lý từng người dùng
grouped = train_filtered.groupby('user_id')

for user_id, group in grouped:
    # Lấy danh sách anime_id và weight của user này
    user_anime_ids = group['anime_id']
    user_weights = group['weight']

    # Lấy index trong tfidf_matrix tương ứng với anime_id
    valid_indices = []
    valid_weights = []

    for anime_id, weight in zip(user_anime_ids, user_weights):
        idx = anime_id_to_index.get(anime_id)
        if idx is not None:
            valid_indices.append(idx)
            valid_weights.append(weight)

    # Nếu user không có anime nào hợp lệ (đã bị lọc) thì bỏ qua
    if not valid_indices:
        continue

    # Lấy các vector TF-IDF của các anime mà user đã xem/thích
    anime_vectors_sparse = tfidf_matrix[valid_indices]

    # Chuyển list weights thành một ma trận cột (sparse) để nhân
    weights_sparse = csr_matrix(valid_weights).T

    # Tính tổng có trọng số: (Vector_Anime * Weight)
    weighted_vectors = anime_vectors_sparse.multiply(weights_sparse)

    # Sum tất cả các vector có trọng số lại
    sum_weighted_vectors = weighted_vectors.sum(axis=0)

    # Tính tổng trọng số (để chia lấy trung bình)
    total_weight = sum(valid_weights)

    if total_weight > 0:
        # Tính vector trung bình (User Profile Vector)
        user_profile_vector = sum_weighted_vectors / total_weight

        # Lưu kết quả
        user_profiles_list.append(csr_matrix(user_profile_vector))
        user_ids_list.append(user_id)

print(f"Đã tính toán xong {len(user_ids_list)} hồ sơ người dùng.")

# 5. Kết quả (User Profile Matrix)
user_profile_matrix = vstack(user_profiles_list)

# Tạo một Series để ánh xạ user_id -> index của hàng trong user_profile_matrix
user_id_to_matrix_index = pd.Series(
    range(len(user_ids_list)),
    index=user_ids_list
)

print(f"Loại ma trận hồ sơ người dùng: {type(user_profile_matrix)}")
print(f"Kích thước ma trận (Shape): {user_profile_matrix.shape}")
print(f"  - Số hàng (User): {user_profile_matrix.shape[0]}")
print(f"  - Số cột (Features): {user_profile_matrix.shape[1]} (khớp với ma trận TF-IDF)")

print("\nMa trận Hồ sơ Người dùng (dạng thưa) - 5 hàng, 50 cột đầu tiên:")
print(user_profile_matrix[:5, :50].toarray())

Đã lọc train_rating: từ 6260012 hàng xuống còn 6074469 hàng (weight > 0).
Đã tính toán xong 73502 hồ sơ người dùng.
Loại ma trận hồ sơ người dùng: <class 'scipy.sparse._csr.csr_matrix'>
Kích thước ma trận (Shape): (73502, 52)
  - Số hàng (User): 73502
  - Số cột (Features): 52 (khớp với ma trận TF-IDF)

Ma trận Hồ sơ Người dùng (dạng thưa) - 5 hàng, 50 cột đầu tiên:
[[0.17636292 0.05013909 0.01174027 0.01390524 0.         0.13951118
  0.         0.07036815 0.05593868 0.21201427 0.10029821 0.04224211
  0.05002971 0.21343588 0.         0.00628478 0.05438604 0.
  0.         0.01586321 0.03836335 0.01390524 0.0181429  0.0031098
  0.01322959 0.00388802 0.01973601 0.01586321 0.         0.0281498
  0.00416686 0.         0.03503307 0.02170938 0.20359315 0.
  0.16894436 0.04224211 0.04685628 0.00821936 0.07493099 0.01586321
  0.         0.03804128 0.00428545 0.03503307 0.17619396 0.
  0.20532139 0.02202438]
 [0.         0.         0.         0.         0.         0.15316157
  0.         0.     

Tạo Gợi ý

In [9]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import time

K = 20
BATCH_SIZE = 2000 # Số lượng user chúng ta xử lý mỗi lần

# Tạo dictionary kết quả cuối cùng
predicted_dict = {}

# 1. Chuẩn bị ma trận 'đã xem' (seen_matrix)
seen_user_ids = train['user_id']
seen_anime_ids = train['anime_id']

# Chuyển đổi IDs sang chỉ số (index) của ma trận
row_indices = [user_id_to_matrix_index.get(uid) for uid in seen_user_ids]
col_indices = [anime_id_to_index.get(aid) for aid in seen_anime_ids]

# Lọc bỏ các cặp không hợp lệ
valid_pairs = [(r, c) for r, c in zip(row_indices, col_indices) if r is not None and c is not None]
valid_row_indices = [pair[0] for pair in valid_pairs]
valid_col_indices = [pair[1] for pair in valid_pairs]

# Tạo ma trận sparse (N_users, N_anime)
# Giá trị 1 tại (i, j) nghĩa là user i đã xem anime j
n_users_in_profile = user_profile_matrix.shape[0]
n_anime_in_profile = tfidf_matrix.shape[0]

seen_matrix = csr_matrix(
    (np.ones(len(valid_row_indices)), (valid_row_indices, valid_col_indices)),
    shape=(n_users_in_profile, n_anime_in_profile),
    dtype=bool
)
print(f"-> Đã tạo ma trận 'đã xem' kích thước {seen_matrix.shape}")

# Chuẩn bị map ánh xạ ngược từ index -> anime_id
index_to_anime_id = anime_processed['anime_id'].to_dict()

# 2. Vòng lặp xử lý bacth
total_users = n_users_in_profile
start_time_total = time.time()

print(f"\nBắt đầu xử lý {total_users} user theo lô (batch_size = {BATCH_SIZE})...")

for i in range(0, total_users, BATCH_SIZE):
    start_time_batch = time.time()

    # Xác định batch tiếp theo
    end_idx = min(i + BATCH_SIZE, total_users)
    print(f"--- Đang xử lý lô user từ {i} đến {end_idx-1} ---")

    # 1. Lấy ra MỘT LÔ user profile
    user_batch_matrix = user_profile_matrix[i:end_idx]

    # 2. Tính toán COSINE SIMILARITY (chỉ cho lô này)
    batch_sim_matrix = cosine_similarity(user_batch_matrix, tfidf_matrix)

    # 3. Lấy phần tương ứng của ma trận "đã xem"
    seen_batch = seen_matrix[i:end_idx]

    # Dùng hàm nonzero() để lấy (hàng, cột) của các anime đã xem và gán điểm tương đồng của chúng về 0
    batch_sim_matrix[seen_batch.nonzero()] = 0.0

    # 4. Xếp hạng và lấy top-K (chỉ cho lô này
    top_k_indices_batch = np.argsort(-batch_sim_matrix, axis=1)[:, :K]

    # 5. Chuẩn bị kết quả (cho lô này)
    for j in range(top_k_indices_batch.shape[0]):
        # Lấy user_id thật
        user_matrix_index = i + j # Chỉ số hàng toàn cục
        user_id = user_ids_list[user_matrix_index]

        # Lấy K chỉ số anime đã được gợi ý
        anime_col_indices = top_k_indices_batch[j]

        # Chuyển đổi K chỉ số này sang anime_id thật
        recommended_anime_ids = [index_to_anime_id[idx] for idx in anime_col_indices]

        # Lưu vào dictionary
        predicted_dict[user_id] = recommended_anime_ids

    end_time_batch = time.time()
    print(f"-> Hoàn thành lô. Thời gian: {end_time_batch - start_time_batch:.2f} giây")

# 3. HOÀN THÀNH
end_time_total = time.time()
print(f"Đã tạo xong dictionary gợi ý (predicted_dict).")
print(f"Tổng số user được gợi ý: {len(predicted_dict)}")
print(f"Tổng thời gian xử lý: {(end_time_total - start_time_total) / 60:.2f} phút")

# Hiển thị ví dụ gợi ý cho 5 user đầu tiên
print("\n--- Ví dụ gợi ý (Top 5 user đầu tiên) ---")
count = 0
for user_id, recs in predicted_dict.items():
    if count >= 5:
        break
    print(f"User {user_id}: {recs}")
    count += 1

-> Đã tạo ma trận 'đã xem' kích thước (73502, 12294)

Bắt đầu xử lý 73502 user theo lô (batch_size = 2000)...
--- Đang xử lý lô user từ 0 đến 1999 ---
-> Hoàn thành lô. Thời gian: 2.07 giây
--- Đang xử lý lô user từ 2000 đến 3999 ---
-> Hoàn thành lô. Thời gian: 1.85 giây
--- Đang xử lý lô user từ 4000 đến 5999 ---
-> Hoàn thành lô. Thời gian: 1.83 giây
--- Đang xử lý lô user từ 6000 đến 7999 ---
-> Hoàn thành lô. Thời gian: 1.79 giây
--- Đang xử lý lô user từ 8000 đến 9999 ---
-> Hoàn thành lô. Thời gian: 1.68 giây
--- Đang xử lý lô user từ 10000 đến 11999 ---
-> Hoàn thành lô. Thời gian: 2.65 giây
--- Đang xử lý lô user từ 12000 đến 13999 ---
-> Hoàn thành lô. Thời gian: 2.39 giây
--- Đang xử lý lô user từ 14000 đến 15999 ---
-> Hoàn thành lô. Thời gian: 2.47 giây
--- Đang xử lý lô user từ 16000 đến 17999 ---
-> Hoàn thành lô. Thời gian: 1.73 giây
--- Đang xử lý lô user từ 18000 đến 19999 ---
-> Hoàn thành lô. Thời gian: 1.78 giây
--- Đang xử lý lô user từ 20000 đến 21999 ---
-> Hoàn

Đánh giá mô hình

In [10]:
# 1. Chuẩn bị ground_truth
# Nhóm theo user_id và tạo một set các anime_id
ground_truth_grouped = test.groupby('user_id')['anime_id'].apply(set)

# Chuyển đổi thành dictionary ground_truth
ground_truth_dict = ground_truth_grouped.to_dict()

print(f"-> Đã tạo ground_truth cho {len(ground_truth_dict)} users có tương tác tích cực trong tập test.")

# 2. Tính toán Metrics
K_eval = 15
print(f"Tổng số user có gợi ý (predicted): {len(predicted_dict)}")
print(f"Tổng số user trong ground_truth: {len(ground_truth_dict)}")
metrics = metrics_eval.evaluate_all(predicted_dict, ground_truth_dict, k=K_eval)

# 3. Phân tích kết quả
print(metrics)


-> Đã tạo ground_truth cho 71102 users có tương tác tích cực trong tập test.
Tổng số user có gợi ý (predicted): 73502
Tổng số user trong ground_truth: 71102
{'Precision@15': 0.032134117183764174, 'Recall@15': 0.04240063364494299, 'MAP@15': 0.02079429456175158}


- Đã chạy lại (7)